<a href="https://colab.research.google.com/github/Diveesha909/BiokindAnalytics/blob/main/OnlineAndOfflineDonations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Available Datsets:

In [ ]:
!ls "/content/drive/My Drive/BioKind-DiaperVillage/Datasets"

AccountsWithoutTransactions.csv  LYBUNT.csv		online_donations.csv  zcta
active_recurring_donations.csv	 offline_donations.csv	tl_2024_us_state      zipcodes_us.csv


# Preprocessing and Imports

## Imports

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

## Generating Donations Dataframe

In [ ]:
# load dataset
offline_df = pd.read_csv('/content/drive/My Drive/BioKind-DiaperVillage/Datasets/offline_donations.csv')

print('Offline Donation Columns:')
print(offline_df.columns)

print('-----------------------------------------------------------------------')

# load dataset
online_df = pd.read_csv('/content/drive/My Drive/BioKind-DiaperVillage/Datasets/online_donations.csv')

print('Online Donation Columns:')
print(online_df.columns)

offline_df['Donation Type'] = 'Offline'
offline_df['online'] = 0

online_df['Donation Type'] = 'Online'
online_df['online'] = 1

online_offline_df = pd.concat([offline_df, online_df], ignore_index=True)
online_offline_df.head()

Offline Donation Columns:
Index(['Donation ID', 'Account ID', 'Shopping Cart ID', 'Campaign ID',
       'Donation Amount', 'Donation Date', 'Account Type', 'Full Name (F)',
       'Company Name', 'Address Line 1', 'City', 'State/Province', 'Zip Code',
       'Campaign Name', 'Purpose', 'Fund', 'Tender Type',
       'Tribute Full Name (F)', 'Unnamed: 18'],
      dtype='object')
-----------------------------------------------------------------------
Online Donation Columns:
Index(['Donation ID', 'Account ID', 'Shopping Cart ID', 'Campaign ID',
       'Donation Amount', 'Donation Date', 'Account Type', 'Full Name (F)',
       'Company Name', 'Address Line 1', 'City', 'State/Province', 'Zip Code',
       'Campaign Name', 'Purpose', 'Fund', 'Tender Type',
       'Tribute Full Name (F)', 'Unnamed: 18'],
      dtype='object')


,Donation ID,Account ID,Shopping Cart ID,Campaign ID,Donation Amount,Donation Date,Account Type,Full Name (F),Company Name,Address Line 1,...,State/Province,Zip Code,Campaign Name,Purpose,Fund,Tender Type,Tribute Full Name (F),Unnamed: 18,Donation Type,online
0,5862,5122,5914,13,595.00,02/23/2023,Company,Jane Braatz,100+ Women Who Care,w10890 Cty Rd V,...,WI,53955,2023 NBC15 Diaper Drive,General Operating,Unrestricted,Check,NaN,NaN,Offline,0
1,8126,7048,8206,17,100.00,02/22/2024,Individual,KATHLEEN A BALTES,NaN,1200 Holtan Road,...,WI,53589,2024 WMTV Diaper Drive,General Operating,Unrestricted,PayPal,NaN,NaN,Offline,0
2,5646,4908,5698,13,25.00,02/23/2023,Individual,Margaret A Listug,NaN,NaN,...,NaN,53589,2023 NBC15 Diaper Drive,General Operating,Unrestricted,PayPal,NaN,NaN,Offline,0
3,7197,6119,7277,17,50.00,02/22/2024,Individual,Becky A Peura,NaN,"2081 Yahara Dr,",...,WI,53589-3364,2024 WMTV Diaper Drive,General Operating,Unrestricted,PayPal,NaN,NaN,Offline,0
4,8985,5934,9064,15,310.89,09/15/2023,Individual,Emily Adams,NaN,141 London Rd,...,WI,53531,2023 Bottoms Up Ball,General Operating,Unrestricted,Other,NaN,NaN,Offline,0


In [ ]:
print('Number of Online Donors:',len(online_df))
print('Number of Offline Donors:', len(offline_df))
print('Total Number of Donors:', len(online_offline_df))
print('Number of Unique Donors:', len(online_offline_df['Account ID'].unique()))

Number of Online Donors: 7253
Number of Offline Donors: 2249
Total Number of Donors: 9502
Number of Unique Donors: 5016


In [ ]:
online_offline_df['Donation Date'] = pd.to_datetime(online_offline_df['Donation Date'])
online_offline_df['Year'] = online_offline_df['Donation Date'].dt.year

online_offline_df['Quarter'] = online_offline_df['Donation Date'].dt.quarter

online_offline_df['Month'] = online_offline_df['Donation Date'].dt.month
online_offline_df['Month Name'] = online_offline_df['Donation Date'].dt.month_name()

online_offline_df['Day'] = online_offline_df['Donation Date'].dt.day
online_offline_df['Day Name'] = online_offline_df['Donation Date'].dt.day_name()

In [ ]:
donation_type_counts = online_offline_df.pivot_table(
    index='Account ID',
    columns='Donation Type',
    values='Donation Amount',
    aggfunc='count',
    fill_value=0
).rename(columns={'Offline': 'Number of Offline Donations', 'Online': 'Number of Online Donations'})

# Merge the counts back to the original summary
donators_df = online_offline_df.groupby('Account ID').agg(
    Total_Donation_Amount=('Donation Amount', 'sum'),
    Number_of_Donations=('Account ID', 'count'),
    Number_of_Offline_Donations=('Donation Type', lambda x: (x == 'Offline').sum()),
    Number_of_Online_Donations=('Donation Type', lambda x: (x == 'Online').sum()),
    Account_Type=('Account Type', 'first'),
).reset_index()

donators_df = donators_df.merge(donation_type_counts, on='Account ID', how='left').fillna(0)

donators_df.head()

,Account ID,Total_Donation_Amount,Number_of_Donations,Number_of_Offline_Donations,Number_of_Online_Donations,Account_Type,Number of Offline Donations,Number of Online Donations
0,5,0.00,1,1,0,Individual,1,0
1,8,100.00,1,0,1,Individual,0,1
2,9,25.00,1,0,1,Individual,0,1
3,12,1105.67,2,0,2,Individual,0,2
4,13,25.00,1,0,1,Individual,0,1


# EDA and Visualizations



## Year-to-year Donation Changes

In [ ]:
yearly_donations = online_offline_df.groupby('Year').agg(
    Total_Donation_Amount=('Donation Amount', 'sum'),
    Total_In_Person_Donation_Amount=('Donation Amount', lambda x: x[online_offline_df['Donation Type'] == 'Offline'].sum()),
    Total_Online_Donation_Amount=('Donation Amount', lambda x: x[online_offline_df['Donation Type'] == 'Online'].sum()),
    Total_Donations=('Account ID', 'count'),
    Number_of_In_Person_Donations=('Donation Type', lambda x: (x == 'Offline').sum()),
    Number_of_Online_Donations=('Donation Type', lambda x: (x == 'Online').sum()),
).reset_index()

# Filter for years between 2020 and 2024
yearly_donations = yearly_donations[(yearly_donations['Year'] >= 2020) & (yearly_donations['Year'] <= 2024)]
yearly_donations.head()

,Year,Total_Donation_Amount,Total_In_Person_Donation_Amount,Total_Online_Donation_Amount,Total_Donations,Number_of_In_Person_Donations,Number_of_Online_Donations
2,2020,62897.22,33851.46,29045.76,394,97,297
3,2021,194248.37,68839.18,125409.19,2401,512,1889
4,2022,279583.25,163138.96,116444.29,2263,602,1661
5,2023,453573.66,340046.85,113526.81,2380,676,1704
6,2024,183673.55,60105.00,123568.55,1914,285,1629


### Yearly Number of Donations by Donation Type(Offline vs Online)

In [ ]:
plot_data = yearly_donations.melt(
    id_vars='Year',
    value_vars=['Number_of_In_Person_Donations', 'Number_of_Online_Donations'],
    var_name='Donation Type',
    value_name='Number of Donations'
)

# Rename donation types for better display
plot_data['Donation Type'] = plot_data['Donation Type'].str.replace(
    'Number_of_', '').str.replace('_Donations', '').str.replace('_', ' ')

# Create the stacked bar chart
fig = px.bar(
    plot_data,
    x='Year',
    y='Number of Donations',
    color='Donation Type',
    title='Number of Donations per Year by Account Type',
    labels={'Number of Donations': 'Donations', 'Year': 'Year'},
    text_auto=True  # Add values on bars
)

# Update layout for better visuals
fig.update_layout(
    barmode='group',
    xaxis_title='Year',
    yaxis_title='Number of Donations',
    legend_title='Donation Type',
    template='plotly_white'
)

# Show the figure
fig.show()

**Analysis**


*   There are more donations done online than in-person every year for the last 5 years.
*   Between 2021-2024 the number of online donations has steadly declined.
*   Between 2021-2023 The number of in-person donations was steadily increasing, but fell significantly in 2024. One explanation of this is that this includes up to November 2024. December fundraising events may significantly contribute to the total number of donations recieved in 2024.




### Yearly Total Amount Donated by Donation Type(Offline vs Online)



In [ ]:
plot_data = yearly_donations.melt(
    id_vars='Year',
    value_vars=['Total_In_Person_Donation_Amount', 'Total_Online_Donation_Amount'],
    var_name='Donation Type',
    value_name='Total Donation Amount'
)

plot_data['Donation Type'] = plot_data['Donation Type'].str.replace(
    'Total_', '').str.replace('_Donation_Amount', '').str.replace('_', ' ')

# Create the stacked bar chart
fig = px.bar(
    plot_data,
    x='Year',
    y='Total Donation Amount',
    color='Donation Type',
    title='Total Donation Amount per Year by Account Type',
    labels={'Number of Donations': 'Donations', 'Year': 'Year'},
    text_auto=True  # Add values on bars
)

# Update layout for better visuals
fig.update_layout(
    barmode='group',
    xaxis_title='Year',
    yaxis_title='Donation Amount(USD)',
    legend_title='Donation Type',
    template='plotly_white'
)

# Show the figure
fig.show()

**Analysis**

* Despite there being more online donations than in-person donations each year, the in-person donations result in a higher total donation amount.
* The total amount earned from online donations has been quite stagnent over the last 4 years. Furthermore, the number of online donations over the last 4 years has also been quite stagnent. This suggests:

  * The online donor user base is consistent.

  * To increase the total amount earned from online donations, the online donor user base needs to be expanded through more email outreach and a larger social media presence.

* The total amount earned from in-person donations grew rapidly between 2020 and 2023. From 2021 to 2023, the number of in-person donations increased at a consistent rate (roughly 15%–20% each year). Meanwhile, during the same period, the total amount earned from in-person donations more than doubled each year. This suggests a potential compounding effect, where an increase in in-person donations encourages donors to contribute more.





### Average Donation Amount per Donation by Year and Account Type

In [ ]:
yearly_donations['Avg_Donation_Amount_Per_Donation'] = yearly_donations['Total_Donation_Amount'] / yearly_donations['Total_Donations']
yearly_donations['Avg_Donation_Amount_Per_In_Person_Donation'] = yearly_donations['Total_In_Person_Donation_Amount'] / yearly_donations['Number_of_In_Person_Donations']
yearly_donations['Avg_Donation_Amount_Per_Online_Donation'] = yearly_donations['Total_Online_Donation_Amount'] / yearly_donations['Number_of_Online_Donations']
yearly_donations.head()

In [ ]:
plot_data = yearly_donations.melt(
    id_vars='Year',
    value_vars=['Avg_Donation_Amount_Per_In_Person_Donation', 'Avg_Donation_Amount_Per_Online_Donation'],
    var_name='Donation Type',
    value_name='Average Donation Amount per Donation(APD)'
)

plot_data['Donation Type'] = plot_data['Donation Type'].str.replace(
    'Avg_Donation_Amount_Per_', '').str.replace('_Donation', '').str.replace('_', ' ')

# Create the stacked bar chart
fig = px.bar(
    plot_data,
    x='Year',
    y='Average Donation Amount per Donation(APD)',
    color='Donation Type',
    title='Average Donation Amount per Donation by Year and Account Type',
    labels={'Number of Donations': 'Donations', 'Year': 'Year'},
    text_auto=True  # Add values on bars
)

# Update layout for better visuals
fig.update_layout(
    barmode='group',
    xaxis_title='Year',
    yaxis_title='Average Donation Amount(USD)',
    legend_title='Donation Type',
    template='plotly_white'
)

fig.add_scatter(
    x=yearly_donations['Year'],
    y=yearly_donations['Avg_Donation_Amount_Per_Donation'],
    mode='lines+markers',  # Line with markers at each year
    name='Total Average Donation',  # Name of the line in the legend
    line=dict(color='black', width=2, dash='dash')  # Line style (black, dashed)
)


# Show the figure
fig.show()

**Analysis**

* Online donations have been relatively consistent, with the exception of 2020. The average amount earned per online donation ranges from approximately \$66 to \$75.
 * The results from 2020 may deviate from the patterns of other years due to the impact of COVID-19.
 * The average online donation amount over the last five years has consistently been below the overall average donation amount for each year. This leads to two conclusions:
   * First, this further highlights the consistency of the online donor base. To increase donation earnings from online donors, more expansive email campaigns and other strategies to grow online engagement are needed.
   * The recommended donation asked from online donators should be in the range of \$66 to \$75.
* The average in-person donation returns an average that is consistently more than double that of the average online donation.

## Year-to-Year Donor Changes

In [ ]:
yearly_unique_donors = online_offline_df.groupby(['Year', 'Account ID']).agg(
    Total_Donation_Amount=('Donation Amount', 'sum'),
    Total_In_Person_Donation_Amount=('Donation Amount', lambda x: x[online_offline_df['Donation Type'] == 'Offline'].sum()),
    Total_Online_Donation_Amount=('Donation Amount', lambda x: x[online_offline_df['Donation Type'] == 'Online'].sum()),
    Total_Number_of_Donations=('Donation Amount', 'count'),
    Total_Number_of_In_Person_Donations=('Donation Type', lambda x: (x == 'Offline').sum()),
    Total_Number_of_Online_Donations=('Donation Type', lambda x: (x == 'Online').sum()),
).reset_index()


yearly_unique_donors = yearly_unique_donors[(yearly_unique_donors['Year'] >= 2020) & (yearly_unique_donors['Year'] <= 2024)]
yearly_unique_donors.head()

## Repeat Donors

In [ ]:
bins = [1, 4, 10, 20, float('inf')]  # Define the ranges: 2-4, 5-10, 10-20, 20+
labels = ['2-4', '5-10', '10-20', '20+']  # Labels for the bins

# Filter out first-time donors
repeat_donors = yearly_unique_donors[yearly_unique_donors['Total_Number_of_Donations'] > 1]

# Categorize donation frequencies into the defined bins
repeat_donors['Donation Frequency Group'] = pd.cut(
    repeat_donors['Total_Number_of_Donations'],
    bins=bins,
    labels=labels,
    right=True  # Include the right boundary in each bin
)
print(repeat_donors.head())

     Year  Account ID  Total_Donation_Amount  Total_In_Person_Donation_Amount  \
107  2020          27                8860.00                           8860.0   
111  2020          78                5100.00                           5000.0   
112  2020         102                 244.00                            100.0   
115  2020         120                 869.59                              0.0   
116  2020         127                1987.47                              0.0   

     Total_Online_Donation_Amount  Total_Number_of_Donations  \
107                          0.00                         11   
111                        100.00                          2   
112                        144.00                         13   
115                        869.59                          6   
116                       1987.47                         10   

     Total_Number_of_In_Person_Donations  Total_Number_of_Online_Donations  \
107                                   11          

<ipython-input-17-f6f5aca54e5c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repeat_donors['Donation Frequency Group'] = pd.cut(


### Number of Repeat Donors over Time
**Donation Frequency** - The number of times a unique donor donated in a year

In [ ]:
# Count the number of donors in each frequency group per year
donor_distribution = repeat_donors.groupby(['Year', 'Donation Frequency Group']).size().reset_index(name='Number_of_Donors')

# Prepare data for plotting
plot_data = donor_distribution

# Create the grouped bar chart
fig = px.bar(
    plot_data,
    x='Year',
    y='Number_of_Donors',
    color='Donation Frequency Group',
    title='Number of Repeat Donors by Donation Frequency Group per Year',
    labels={'Number_of_Donors': 'Number of Donors', 'Year': 'Year'},
    text_auto=True  # Display the counts on bars
)

# Update layout for better visuals
fig.update_layout(
    barmode='group',  # Group bars by frequency group
    xaxis_title='Year',
    yaxis_title='Number of Donors',
    legend_title='Donation Frequency Group',
    template='plotly_white'
)

# Show the figure
fig.show()

<ipython-input-47-a5d105e88187>:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



**Analysis**

* **2-4 Donations/Low Frequency Category**:
The 2-4 times a year (Low Frequency) category consistently represents the largest demographic of repeated donors. This group consistently accounts for more than 50% of the total repeated donors each year. The prevalence of this category suggests that a significant portion of donors donate sporadically but consistently return multiple times within a year.

* **10-20 Donations/High Frequency Category**:
Surprisingly, the 10-20 donations a year (High Frequency) category is the second-largest demographic of repeated donors. This group has shown remarkable consistency over the years, maintaining a strong presence in terms of donor count. This suggests that a small but dedicated group of donors is actively supporting the cause with frequent contributions, providing steady support year after year.

* **5-10 Donations/Medium Frequency Category**:
The 5-10 donations a year (Medium Frequency) category displays some fluctuation over the years, with no clear upward or downward trend. This group appears to be less consistent than both the Low and High Frequency categories.

* **20+ Donations/Extremely High Frequency Category**:
The 20+ donations a year (Extremely High Frequency) category is rare, comprising a small fraction of the total donor pool. This group has remained minimal in size, with little to no significant growth, indicating that while some highly dedicated donors contribute frequently, they represent a very small subset of the total donor base.



### Percent Change of Number of Repeat Donors over Time

In [ ]:
# Calculate percentage of donors for each frequency group per year
donor_distribution['Percent_of_Total'] = donor_distribution.groupby('Year')['Number_of_Donors'].transform(lambda x: x / x.sum() * 100)

# Prepare data for plotting (we can keep it as it is since it's already structured)
plot_data = donor_distribution

# Create a subplot with multiple pie charts (one for each year)
fig = make_subplots(
    rows=1,  # One row of pie charts
    cols=len(plot_data['Year'].unique()),  # Number of columns = number of unique years
    subplot_titles=[f'{year}' for year in plot_data['Year'].unique()],
    specs=[[{'type': 'pie'}] * len(plot_data['Year'].unique())]  # Create pie charts in all columns
)

# Add a pie chart for each year
for idx, year in enumerate(plot_data['Year'].unique()):
    year_data = plot_data[plot_data['Year'] == year]

    # Create pie chart for each year
    pie_chart = go.Pie(
        labels=year_data['Donation Frequency Group'],
        values=year_data['Percent_of_Total'],
        name=f'{year}'
    )

    # Add the pie chart to the appropriate subplot
    fig.add_trace(pie_chart, row=1, col=idx + 1)

# Update layout for better visuals
fig.update_layout(
    title='Percentage of Donors by Frequency Group for Each Year',
    showlegend=True,
    template='plotly_white'
)

# Show the figure
fig.show()

### Total Amount Donated from Repeat Donors over Time

### Average Amount Donated from Repeat Donors over Time

# Regression Analysis

In [ ]:
import statsmodels.api as sm

## Demographic Trends

### Year to Year Trends

In [ ]:
# Count donation types grouped by Account ID and Year
donation_type_counts = online_offline_df.pivot_table(
    index=['Account ID', 'Year'],  # Group by both Account ID and Year
    columns='Donation Type',
    values='Donation Amount',
    aggfunc='count',
    fill_value=0
).rename(columns={'Offline': 'Number of Offline Donations', 'Online': 'Number of Online Donations'})

# Group by Account ID and Year for totals
yearly_donators_df = online_offline_df.groupby(['Account ID', 'Year']).agg(
    Total_Donation_Amount=('Donation Amount', 'sum'),
    Number_of_Donations=('Account ID', 'count'),
    Number_of_Offline_Donations=('Donation Type', lambda x: (x == 'Offline').sum()),
    Number_of_Online_Donations=('Donation Type', lambda x: (x == 'Online').sum()),
    Account_Type=('Account Type', 'first')
).reset_index()

# Filter to include only years from 2020 to 2024
yearly_donators_df = yearly_donators_df.query("2020 <= Year <= 2024")

# Merge the donation type counts with the yearly summary
yearly_donators_df = yearly_donators_df.merge(
    donation_type_counts,
    on=['Account ID', 'Year'],  # Merge on both Account ID and Year
    how='left'
).fillna(0)

# Display the first few rows of the filtered result
print('Number of Rows:', len(yearly_donators_df))
yearly_donators_df.head()



Number of Rows: 7293


,Account ID,Year,Total_Donation_Amount,Number_of_Donations,Number_of_Offline_Donations,Number_of_Online_Donations,Account_Type,Number of Offline Donations,Number of Online Donations
0,8,2021,100.00,1,0,1,Individual,0,1
1,12,2021,1105.67,2,0,2,Individual,0,2
2,15,2020,30.00,1,1,0,Individual,1,0
3,27,2020,8860.00,11,11,0,Company,11,0
4,27,2021,4217.32,4,4,0,Company,4,0


#### Unique Donors over Time by Donation Type(Online vs Offline)

In [ ]:
import plotly.graph_objects as go

# Group by Year and sum up the Offline and Online Donations
yearly_donators_df_grouped = yearly_donators_df.groupby('Year').agg(
    Total_Offline_Donations=('Number_of_Offline_Donations', 'sum'),
    Total_Online_Donations=('Number_of_Online_Donations', 'sum'),
    Total_Donations=('Number_of_Donations', 'sum'),
).reset_index()

# Create the bar chart for Offline and Online Donations
fig = go.Figure()

# Add bars for Offline donations
fig.add_trace(
    go.Bar(
        x=yearly_donators_df_grouped['Year'],
        y=yearly_donators_df_grouped['Total_Offline_Donations'],
        name='Offline Donations',
        marker_color='orange',
    )
)

# Add bars for Online donations
fig.add_trace(
    go.Bar(
        x=yearly_donators_df_grouped['Year'],
        y=yearly_donators_df_grouped['Total_Online_Donations'],
        name='Online Donations',
        marker_color='blue',
    )
)

# Add a trend line for the total donations
fig.add_trace(
    go.Scatter(
        x=yearly_donators_df_grouped['Year'],
        y=yearly_donators_df_grouped['Total_Donations'],
        mode='lines+markers',
        name='Trend Line (Total Donations)',
        line=dict(color='green', dash='dash'),
        marker=dict(size=8, color='green'),
    )
)

# Update layout and axis titles
fig.update_layout(
    title='Offline and Online Donations Over the Years (2020-2024)',
    xaxis_title='Year',
    yaxis_title='Total Number of Donators',
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    template='plotly_white',
    bargap=0.1,  # Decrease the space between bars; smaller = thinner bars
    barmode='stack',  # Stack the bars for each year
    width=800
)

# Show the figure
fig.show()
